In [92]:
# Imports
import requests
from bs4 import BeautifulSoup
import psycopg2

In [93]:
# Variablen
conn_Ok = True


In [94]:
# Check if connection to DB possible
try:
    connection = psycopg2.connect(
        dbname='PSQL_ADSFS2023Gruppe15',
        user='ADSFS2023Gruppe15',
        password='ADS_FS_2023_G15!?',
        host='localhost',
        port='5432'
    )

except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
        conn_Ok = False

In [95]:
# Funktionen

def connect_and_save_data(matchday, team_name, value, data_type):
    try:
        cursor = connection.cursor()

        if team_name == 'Bor. Mönchengladbach':
            team_name = 'Borussia Mönchengladbach'
        elif team_name == '1899 Hoffenheim':
            team_name = 'TSG 1899 Hoffenheim'
        # Vergleicht Team Namen und sucht nach ID
        cursor.execute("SELECT team_id FROM bundesliga_mannschaften WHERE mannschaft = %s", (team_name,))
        result = cursor.fetchone()[0]
        print(result, team_name)
        if result:
            team_id = result
        else:
           print(f'Keine Übereinstimmung für {team_name} gefunden')

        # Füge die extrahierten Daten in die Tabelle ein
        if data_type == 'pass':
            insert_data_query = '''
                INSERT INTO bundesliga_pass_stats (matchday, team_name, pass_percentage) 
                VALUES (%s, %s, %s);
            '''
        elif data_type == 'schuesse':
            insert_data_query = '''
                INSERT INTO bundesliga_shots_stats (matchday, team_name, value)
                VALUES (%s, %s, %s);
            '''
        elif data_type == 'zweikampf':
            insert_data_query = '''
                INSERT INTO bundesliga_duels_stats (matchday, team_name, value)
                VALUES (%s, %s, %s);
            '''
        if data_type == 'pass':
            cursor.execute(insert_data_query, (matchday, team_id, float(value.replace(',', '.'))))
        else:
            cursor.execute(insert_data_query, (matchday, team_id, value))
                        
       #connection.commit()
    except:
        connection.rollback()
        raise

In [96]:
if conn_Ok:
    # Anzahl Pässe für die spielTage im angegebenen Range
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-paesse/'

        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Suche nach allen Mannschaftsnamen und Passquoten
            team_elements = soup.find_all('td', class_='team-name team-name-list')
            pass_percentage_elements = soup.find_all('td', class_='team_stats-passes_complete_percentage team_stats-passes_complete_percentage-format')

            # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
            for team_element, pass_percentage_element in zip(team_elements, pass_percentage_elements):
                team_name = team_element.text.strip()
                pass_percentage = pass_percentage_element.text.strip()

                # Speichere die Daten in der PostgreSQL-Datenbank
                connect_and_save_data(matchday, team_name, pass_percentage, 'pass')

        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday}')

65 1. FC Köln
87 Borussia Mönchengladbach
40 FC Bayern München
1635 RB Leipzig
6 Bayer Leverkusen
129 VfL Bochum
95 FC Augsburg
131 VfL Wolfsburg
7 Borussia Dortmund
54 Hertha BSC


TypeError: 'NoneType' object is not subscriptable

In [ ]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-torschuesse/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        shots_elements = soup.find_all('td', class_='team_stats-shots team_stats-shots-list')

        for team_element, shots_element in zip(team_elements, shots_elements):
            team_name = team_element.text.strip()
            shots_value = shots_element.text.strip()

            connect_and_save_data(matchday, team_name, shots_value, 'schuesse')

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')

In [ ]:
for matchday in range(1, 35):
    url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-zweikaempfe/'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        team_elements = soup.find_all('td', class_='team-name team-name-list')
        duels_elements = soup.find_all('td', class_='team_stats-zweikaempfe team_stats-zweikaempfe-format')

        for team_element, duels_element in zip(team_elements, duels_elements):
            team_name = team_element.text.strip()
            duels_value = duels_element.text.strip()

            connect_and_save_data(matchday, team_name, duels_value, 'zweikampf')

    else:
        print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Zweikämpfe')

KeyboardInterrupt: 